#Import Library

In [ ]:
import urllib.request
import json
import sys
import os
import glob
import pandas as pd



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Donwload Query

In [ ]:
import os
import json
import urllib.request
import pandas as pd

# Tentukan direktori lokal untuk menyimpan data
corepath = '/content/drive/MyDrive/DL/data'  # Anda bisa mengganti ini dengan path yang sesuai di komputer Anda

def save_json(searchTerms, birdName, country):
    numPages = 1
    page = 1
    path = os.path.join(corepath, birdName.replace(':', '')) + "/"

    if not os.path.exists(path):
        print("Creating subdirectory " + path + " for downloaded files...")
        os.makedirs(path)

    while page <= numPages:
        print("Loading page " + str(page) + "...")
        url = 'https://www.xeno-canto.org/api/2/recordings?query={0}&page={1}'.format(searchTerms.replace(' ', '%20'), page)
        print("URL yang digunakan:", url)  # Memeriksa URL yang digunakan

        try:
            jsonPage = urllib.request.urlopen(url)
            jsondata = json.loads(jsonPage.read().decode('utf-8'))
            filename = os.path.join(path, birdName.replace(':', '') + "_jsondata_p" + str(page) + ".json")

            with open(filename, 'w') as outfile:
                json.dump(jsondata, outfile)
            print(f"Saved JSON data to {filename}")

            numPages = jsondata['numPages']
            page += 1
        except Exception as e:
            print(f"Error fetching data: {e}")
            break  # Keluar dari loop jika terjadi kesalahan

    print("Found ", numPages, " pages in total.")
    return path

def read_data(searchTerm, path, birdName):
    data = []
    numPages = 1
    page = 1

    while page <= numPages:
        json_file_path = os.path.join(corepath, path, birdName.replace(':', '') + "_jsondata_p" + str(page) + ".json")
        print("Trying to open:", json_file_path)

        if not os.path.exists(json_file_path):
            print(f"File not found: {json_file_path}")
            return data

        with open(json_file_path, 'r') as jsonfile:
            jsondata = json.load(jsonfile)

        numPages = jsondata['numPages']
        for recording in jsondata['recordings']:
            data.append(recording[searchTerm])

        page += 1

    return data

def download(searchTerms, birdName, country):
    path = save_json(searchTerms, birdName, country)
    filenamesID = read_data('id', path, birdName)
    fileaddress = read_data('file', path, birdName)
    numfiles = len(filenamesID)

    print("A total of ", numfiles, " files will be downloaded")

    max_files_to_download = 200  # Batas maksimal file yang akan diunduh
    total_downloaded = min(numfiles, max_files_to_download)  # Menghitung jumlah file yang akan diunduh

    records = []
    for i in range(total_downloaded):
        print("Saving file ", i + 1, "/", total_downloaded, os.path.join(path, filenamesID[i] + birdName.replace(':', '') + ".wav"))
        complete_url = fileaddress[i]

        try:
            urllib.request.urlretrieve(complete_url,
                                       os.path.join(corepath, path, filenamesID[i] + birdName.replace(':', '') + ".wav"))
            records.append({
                'id': filenamesID[i],
                'file': complete_url,
                'species': birdName
            })
        except Exception as e:
            print(f"Error downloading file {filenamesID[i]}: {e}")

    # Create a pandas dataframe of records & convert to .csv file
    record_df = pd.DataFrame(records)
    record_df.to_csv(os.path.join(corepath, birdName.replace(' ', '') + ".csv"), index=False)

    print("===============Done Download============")

# Daftar spesies yang ingin diunduh
species_list = [
    "Pitta sordida",
    "Dryocopus javensis",
    "Caprimulgus macrurus",
    "Pnoepyga pusilla",
    "Malacocincla malaccensis",
    "Anthipes solitaris",
    "Buceros rhinoceros"
]

# Loop untuk mengunduh setiap spesies
for species in species_list:
    print(f"Downloading recordings for {species}...")
    download(species, species, "Indonesia")  # Ganti dengan negara yang sesuai jika diperlukan

Creating subdirectory /content/drive/MyDrive/DL/data/Pitta sordida/ for downloaded files...
Loading page 1...
URL yang digunakan: https://www.xeno-canto.org/api/2/recordings?query=Pitta%20sordida&page=1
Saved JSON data to /content/drive/MyDrive/DL/data/Pitta sordida/Pitta sordida_jsondata_p1.json
Found  1  pages in total.
Trying to open: /content/drive/MyDrive/DL/data/Pitta sordida/Pitta sordida_jsondata_p1.json
Trying to open: /content/drive/MyDrive/DL/data/Pitta sordida/Pitta sordida_jsondata_p1.json
A total of  178  files will be downloaded
Saving file  1 / 178 /content/drive/MyDrive/DL/data/Pitta sordida/937759Pitta sordida.wav
Saving file  2 / 178 /content/drive/MyDrive/DL/data/Pitta sordida/935587Pitta sordida.wav
Saving file  3 / 178 /content/drive/MyDrive/DL/data/Pitta sordida/935277Pitta sordida.wav
Saving file  4 / 178 /content/drive/MyDrive/DL/data/Pitta sordida/908991Pitta sordida.wav
Saving file  5 / 178 /content/drive/MyDrive/DL/data/Pitta sordida/900867Pitta sordida.wav


#Convert Meta Data

In [ ]:
import os
import pandas as pd
import glob

# Tentukan direktori kerja
corepath = '/content/drive/MyDrive/DL/data'
os.chdir(corepath)  # Ganti direktori kerja ke folder data

# Step 1: Menggabungkan metadata CSV
extension = 'csv'  # Semua file metadata disimpan dalam format CSV
all_metadata_files = [i for i in glob.glob(f'**/*.{extension}', recursive=True)]

# Fungsi untuk mengubah CSV metadata menjadi DataFrame
def load_metadata(file):
    try:
        data = pd.read_csv(file)  # Baca CSV
        return data  # Kembalikan DataFrame
    except Exception as e:
        print(f"Error loading {file}: {e}")
        return pd.DataFrame()  # Kembalikan DataFrame kosong jika terjadi kesalahan

# Menggabungkan semua metadata ke dalam satu DataFrame
all_metadata = pd.concat([load_metadata(f) for f in all_metadata_files], ignore_index=True)

# Simpan ke file CSV gabungan
combined_csv_path = "burung_metadata.csv"
all_metadata.to_csv(combined_csv_path, index=False, encoding='utf-8-sig')
print(f"==========Done Combine Metadata to CSV: {combined_csv_path}==========")

==========Done Combine Metadata to CSV: burung_metadata.csv==========


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/DL/data/burung_metadata.csv', delimiter=',')

In [ ]:
df.head()

,id,file,species
0,937759,https://xeno-canto.org/937759/download,Pitta sordida
1,935587,https://xeno-canto.org/935587/download,Pitta sordida
2,935277,https://xeno-canto.org/935277/download,Pitta sordida
3,908991,https://xeno-canto.org/908991/download,Pitta sordida
4,900867,https://xeno-canto.org/900867/download,Pitta sordida


In [ ]:
len(df)

895

In [ ]:
df.describe()

,id
count,895.000000
mean,468124.710615
std,288405.589881
min,18705.000000
25%,205051.000000
50%,439605.000000
75%,758850.000000
max,951547.000000


In [ ]:
df['en'].value_counts()

,count
en,
Great Argus,92
Helmeted Hornbill,80
Sumatran Laughingthrush,27


In [ ]:
sum(df['en'].value_counts() >= 100) / df['en'].nunique()

0.0

In [ ]:
sum(df['en'].value_counts() <= 100) / df['en'].nunique()

1.0

In [ ]:
df['q'].value_counts()

,count
q,
A,79
B,66
C,49
D,3
E,2


In [ ]:
df.shape

(1094, 39)

In [ ]:
df[df.q.isin(['D', 'E'])]

,id,gen,sp,ssp,group,en,rec,cnt,loc,lat,...,bird-seen,animal-seen,playback-used,temp,regnr,auto,dvc,mic,smp,species
78,169444,Rhinoplax,vigil,NaN,birds,Helmeted Hornbill,Johannes Fischer,Indonesia,NaN,NaN,...,no,no,no,NaN,NaN,no,NaN,NaN,44100.0,NaN
79,304715,Rhinoplax,vigil,NaN,birds,Helmeted Hornbill,dick balsamo,Thailand,NaN,NaN,...,yes,yes,unknown,NaN,NaN,no,NaN,NaN,44100.0,NaN
169,314952,Argusianus,argus,NaN,birds,Great Argus,Bobby Wilcox,Thailand,NaN,NaN,...,no,no,no,NaN,NaN,no,NaN,NaN,44100.0,NaN
170,169536,Argusianus,argus,NaN,birds,Great Argus,Johannes Fischer,Indonesia,NaN,NaN,...,no,no,no,NaN,NaN,no,NaN,NaN,44100.0,NaN
171,173116,Argusianus,argus,argus,birds,Great Argus,Frank Lambert,Malaysia,NaN,NaN,...,no,no,no,NaN,NaN,no,NaN,NaN,44100.0,NaN


delete audio with q= D & E manually

In [ ]:
df1 = df.drop([78, 79, 169, 170, 171])

In [ ]:
df1.to_csv(corepath + 'birdmetadata_2.csv')

#Total of .wav files

In [ ]:
corepath = '/content/drive/MyDrive/DL/data'
blist=[]
for root, dirs, files in os.walk(corepath):
    for file in files:
        if file.endswith(".wav"):
            blist.append(os.path.join(root, file))

print('Found ', len(blist),' files.')

Found  1185  files.
